<a href="https://colab.research.google.com/github/EnricH67/Base-de-dades/blob/main/Copia_de_Extract_Text_from_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Text from PDF Files

Let's look at how to extract text from a PDF file, using the [`pdfx`](https://www.metachris.com/pdfx/) library in Python.

First we need to install the library:

In [1]:
!pip install pdfx

     |████████████████████████████████| 178 kB 7.0 MB/s 
     |████████████████████████████████| 5.6 MB 28.3 MB/s 
     |████████████████████████████████| 4.0 MB 37.0 MB/s 
  Attempting uninstall: chardet
    Found existing installation: chardet 3.0.4
    Uninstalling chardet-3.0.4:
      Successfully uninstalled chardet-3.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires chardet<4,>=3.0.2, but you have chardet 4.0.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Next, let's work with an example from the corpus in the [Rich Context leaderboard competition](https://github.com/Coleridge-Initiative/rclc/blob/master/corpus.ttl) – a machine learning competition about parsing named entities from PDFs of open access research publications.

The following snippets in [TTL format](https://en.wikipedia.org/wiki/Turtle_(syntax)) show a research paper `publication-7aa3d69253e37668541c` hosted on [EuropePMC](https://europepmc.org/) that has a known link to a dataset `dataset-0a7b604ab2e52411d45a` hosted by the [Centers for Disease Control and Prevention](https://wwwn.cdc.gov/nchs/nhanes/).

```
:publication-7aa3d69253e37668541c
  rdf:type :ResearchPublication ;
  foaf:page "http://europepmc.org/articles/PMC3001474"^^xsd:anyURI ;
  dct:publisher "PLoS One" ;
  dct:title "VKORC1 common variation and bone mineral density in the Third National Health and Nutrition Examination Survey" ;
  dct:identifier "10.1371/journal.pone.0015088" ;
  :openAccess "http://europepmc.org/articles/PMC3001474?pdf=render"^^xsd:anyURI ;
  cito:citesAsDataSource :dataset-0a7b604ab2e52411d45a ;
.

:dataset-0a7b604ab2e52411d45a
  rdf:type :Dataset ;
  foaf:page "https://wwwn.cdc.gov/nchs/nhanes/"^^xsd:anyURI ;
  dct:publisher "Centers for Disease Control and Prevention" ;
  dct:title "National Health and Nutrition Examination Survey" ;
  dct:alternative "NHANES" ;
  dct:alternative "NHANES I" ;
  dct:alternative "NHANES II" ;
  dct:alternative "NHANES III" ;
.
```

The paper is:

  * ["VKORC1 common variation and bone mineral density in the Third National Health and Nutrition Examination Survey"](http://europepmc.org/articles/PMC3001474); Dana C. Crawford, Kristin Brown-Gentry, Mark J. Rieder; _PLoS One_. 2010; 5(12): e15088.

We'll used `pdfx` to download the PDF file directly from the open access URL:

In [ ]:
import pdfx

pdf = pdfx.PDFx("http://europepmc.org/articles/PMC3001474?pdf=render")

pdf

Next, use the `get_text()` function to extract the text from the `pdf` object:

In [ ]:
text = pdf.get_text()
text

Now we can use `spaCy` to parse that text:

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

Let's look at a dataframe of the parsed tokens:

In [ ]:
import pandas as pd

cols = ("text", "lemma", "POS", "explain", "stopword")
rows = []

for t in doc:
    row = [t.text, t.lemma_, t.pos_, spacy.explain(t.pos_), t.is_stop]
    rows.append(row)

df = pd.DataFrame(rows, columns=cols)
df

The parsed text shows lots of characters that could be cleaned up, but for this demo, let's run *named entity resolution* in `spaCy` to extract the entities:

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Great, that identified multiple mentions of the _NHANES_ dataset:

  * `the Third National Health and Nutrition Examination Survey` _ORG_
  * `NHANES III` _PERSON_
  
The default labels aren't correct, but we could [update the Named Entity Recognizer](https://spacy.io/usage/training#ner) in `spaCy` to fix that.